<a href="https://colab.research.google.com/github/myazann/Lung_Cancer/blob/main/LungCancer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git init
!git pull https://github.com/myazann/TCIA-API-SDK.git

!cp -r /content/tcia-rest-client-python/src/* /content

!rm -rf tcia-rest-client-java/
!rm -rf tcia-rest-client-python/

!cp /content/drive/MyDrive/Annotation.zip /content
!unzip Annotation.zip
!rm Annotation.zip

!pip install xmltodict
!rm -rf sample_data
!rm sample.py
!rm README.md

from tciaclient import TCIAClient
import json
import os
import numpy as np
from xml.etree import cElementTree as ElementTree
import xmltodict
import urllib.request, urllib.error, urllib.parse
import subprocess

tc = TCIAClient(baseUrl="https://services.cancerimagingarchive.net/services/v4", resource = "TCIA")

In [ ]:
folders = np.array(os.listdir("Annotation"))
##folders = [i for i in folders if "A" in i]

bboxs = {}

for folder in folders:
  path = "Annotation/" + folder
  for img in os.listdir(path):
    file_path = path + "/" + img
    try:
      dt = xmltodict.parse(ElementTree.tostring(ElementTree.parse(file_path).getroot()))["annotation"]["object"]
      if isinstance(dt, list):
        box_list = []
        for box in dt:
          box_list.append(box)
      else:
        dt = dt["bndbox"]
      img_name = folder + "-" + img.split(".x")[0]
      bboxs[img_name] = dt
    except Exception as e:
      print(e)

with open('bboxs.json', "w") as json_file:
  json.dump(bboxs, json_file)

clctn = "Lung-PET-CT-Dx"
series = json.loads(tc.get_series(collection = clctn, modality = "CT").read())

ptnt_to_series_ids = {}

for elem in series:
  try:
    if elem["BodyPartExamined"] == "CHEST":
      pt_id = elem["PatientID"].split("-")[1]
      ptnt_to_series_ids[elem["SeriesInstanceUID"]] = pt_id
  except Exception as e:
    print(e)

sop_to_series_ids = {}

for series in ptnt_to_series_ids.keys():
  sop = json.loads(tc.get_SOP_instance(series).read())
  for id in sop:
    sop_to_series_ids[id["SOPInstanceUID"]] = series

imgs_with_bboxs = [box.split("-")[1] for box in bboxs.keys()]
sop_to_series_ids = dict((img, sop_to_series_ids[img]) for img in sop_to_series_ids.keys() if img in imgs_with_bboxs)

with open('ptnt_to_series_ids.json', 'w') as json_file:
  json.dump(ptnt_to_series_ids, json_file)

with open('sop_to_series_ids.json', 'w') as json_file:
  json.dump(sop_to_series_ids, json_file)

In [ ]:
if not os.path.exists("images"):
  os.mkdir("images")
  if os.path.exists("./drive/MyDrive/images.zip"):
    subprocess.call(["cp","./drive/MyDrive/images.zip","."])
    subprocess.call(["unzip","images.zip","-d","images"])

existing_images = [img.split(".d")[0] for img in os.listdir("images")]
sop_to_series_ids = dict((img, sop_to_series_ids[img]) for img in sop_to_series_ids.keys() if img not in existing_images)

for img in sop_to_series_ids.keys():
  download_path = "./images/" + img + ".dcm"
  tc.get_single_image(sop_to_series_ids[img], img, downloadPath = download_path)
  if len(os.listdir("images")) % 1000 == 0:
    print("Saving images!")
    subprocess.call(["zip","-r","images.zip","images"])
    subprocess.call(["mv","images.zip","./drive/MyDrive/"])


subprocess.call(["zip","-r","images.zip","images"])
subprocess.call(["mv","images.zip","./drive/MyDrive/"])

In [82]:
!cp ./drive/MyDrive/ptnt_to_series_ids.json .
!cp ./drive/MyDrive/sop_to_series_ids.json .
!cp ./drive/MyDrive/bboxs.json .
!cp ./drive/MyDrive/images.zip .
!mkdir images
!unzip images.zip -d images
!rm images.zip

with open('ptnt_to_series_ids.json') as json_file:
  ptnt_to_series_ids = json.load(json_file)

with open('sop_to_series_ids.json') as json_file:
  sop_to_series_ids = json.load(json_file)

with open('bboxs.json') as json_file:
  bboxs = json.load(json_file)

In [89]:
classes = [cls[0] for cls in bboxs.keys()]

In [91]:
import pandas as pd

pd.Series(classes).value_counts(normalize = True)

A    0.661988
G    0.232914
B    0.098729
E    0.006369
dtype: float64